[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantificAid/hands-on-data-science-example/blob/master/hands-on-data-science-example.ipynb)

# Data Science zum Mitmachen: Wetter, Radfahrer und mehr...

## Einleitung

Wie wir in der Bastelstrecke von [diy-iot2ds](https://github.com/birds-on-mars/diy-iot2ds) lernen können, ist es gar nicht sooo schwer und aufwendig, Wetterdaten automatisch zu erfassen. Allerdings ist die Analyse der isolierten Daten etwas ... gähn. Um spannendere Dinge zu tun und zu erkennen, muss man schon ein paar verschiedene Daten zusammenbringen.

Und auch das ist machbar! **Einen kleinen Teil des Ganzen muss/darf der geneigte Leser dabei selber machen.**
Google bietet eine kostenlose Umgebung dafür an. Falls noch nicht geschehen, jetzt **bitte einfach oben den Button "open in colab" anklicken** (und nach Studie der Bedingungen von Google diese ggf. akzeptieren, damit die Kiste läuft). 

Um dies zu veranschaulichen haben wir uns von Jake van der Plas, einem Astrophysiker, Data Science-Meister und Python-Guru inspirieren lassen. Jake hält nicht nur viele Talks zum Thema und hat hervorragende Bücher wie das [Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) geschrieben, das es übrigens komplett open-source und interaktiv durch- und zu bearbeiten gibt. Er schreibt auch seit längerem einen Blog, in dem er auch immer wieder Methoden anschaulich beschreibt.  

In 2014 hat er dort die Frage gestellt, ob es einen Aufschwung in der Nutzung von Fahrrädern in Seattle gibt. Hier der [Link zum Blog von dunnemals](https://jakevdp.github.io/blog/2014/06/10/is-seattle-really-seeing-an-uptick-in-cycling/).  

**Hier werden öffentlich zugängliche und recht volatile Daten zum (automatisch erfassten) Fahrrad-Verkehr über die Fremont-Brücke in Seatlle mit Wetter- und auch berechneten Daten in Verbindung gebracht, um zu verifizieren, ob es tatsächlich eine Veränderung der Fahrrad-Fahrten im Laufe der Zeit gibt bzw. welchen Einfluss andere Umgebungsdaten haben.** 

Die Antwort von Jake van der Plas in seinem Blog damals war: ja, es gibt einen Zuwachs an Fahrradverkehr.
Jetzt sind wir ein paar Jahre weiter und wir fragen uns daher: ist das immer noch der Fall?  

Let's see...

## Verwendete Datenquellen

Auf den Fahrradwegen der Fremont-Brücke in Seattle werden mit magnetischen Induktionsschleifen automatisch Fahrräder gezählt. Da [Open-Data](https://de.wikipedia.org/wiki/Open_Data) in den USA auch bei öffentlichen Institutionen recht verbreitet ist, sind diese Daten frei zugänglich. Sie werden [auf dem Open-Data-Portal der Stadt Seattle bereitgestellt](https://data.seattle.gov/Transportation/Fremont-Bridge-Hourly-Bicycle-Counts-by-Month-Octo/65db-xm6k). Die hier im Repository hinterlegten Daten sind vom 04. Juni 2019 und reichen zurück bis 2012.  

So sieht übrigens die Fremont-Brücke in Seattle aus:

![Fremont-Brücke in Seatlle](https://upload.wikimedia.org/wikipedia/commons/5/50/Seattle_%E2%80%94_Fremont_Bridge_%E2%80%94_%282016-06-12%29%2C_01.jpg)  

Auch Wetterdaten aus den USA sind frei zugänglich über das [National Center of Environmental Information](://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND), wo u.a. die Wetterstation des SeaTac Airport in Seattle ausgewählt werden kann (ID der Wetterstation "USW00024233"). Hier haben wir uns die letzten verfügbaren Daten vom 25. Juni 2019 (auch zurück bis 2012) besorgt.

Die [Daten liegen bereits in diesem Repository](https://github.com/QuantificAid/hands-on-data-science-example/blob/master/Daten/):

In [9]:
Pfad_zu_Daten = 'https://raw.githubusercontent.com/QuantificAid/hands-on-data-science-example/master/Daten/'

## Vorgehen

Auch hier werden wir die üblichen Schritte einer Data Science-Analyse durchlaufen:
- **Laden und erstes Explorieren der Daten** (*Gathering*) verbunden mit einer ersten ganz kurzen ersten Exploration, wie die Daten in der Rohversion aussehen.
- **Strukturieren der Daten** (*Preprocessing*) um die Daten in eine gesamthaft weiterverarbeitbare Form zu bringen.
- **Bereinigung und weitere Exploration der Daten** *(Cleansing, Exploration)*, um die Daten in die finale Form für die Modellbildung zu bringen. Glücklicherweise sind die Daten recht "sauber", was den ansonsten oft sehr mühseligen Teil der Bereinigung sehr kurz macht. Wir werden die Daten aber recht ausgiebig visualisieren.
- Last, but not least **Modellierung der Daten** *(Modelling)*. Hier benutzen wir werden wir eine "einfache" multi-lineare Regression anwenden, um zu prüfen, welcher Einflussfaktor wie (linear) wirkt und welcher Trend sich am Ende für den zeitlichen Verlauf der Fahrrad-Nutzung ergibt. Wie lineare Regression funktioniert wird im entsprechenden Teil erläutert.

## Verwendete Programmbibliotheken

Die Analyse der Daten nehmen wir in der Programmiersprache Python vor, die für sehr vieles geeignet ist, aber auch sehr beliebt für Data Science, Machine Learning und AI ist.

Sie ist nicht nur deshalb in diesen Gebieten so beliebt, weil sie relativ leicht zu lernen und sehr vielseitig ist, sondern auch und insbesondere, weil es viele schlaue Geister in Python sehr coole Programm-Bibliotheken geschrieben haben, die open source bereitstehen, deren enthaltene Objekte, Methoden und Funktionen sofort verwendet werden können und die einem viel Arbeit abnehmen.

Wir laden uns die folgenden Bibliotheken:
- [pandas](https://pandas.pydata.org/) zur effizienten Bearbeitung von Datentabellen
- [NumPy](https://www.numpy.org/) zur schnellen numerischen Bearbeitung von Tabellendaten und Matrizen (wird im Hintergrund auch von pandas verwendet)
- [altair](https://altair-viz.github.io/) zur grafischen und interaktiven Darstellung von Daten und
- [scikit-learn oder auch sklearn](https://scikit-learn.org/stable/), einer elegant programmierten Bibliothek mit vielen Funktionen zum maschinellen Lernen (u.a. der linearen Regression).

In [10]:
# Importieren von Programm-Bibliotheken

# Programm-Bibliothek zur effizienten Bearbeitung von Datentabellen
import pandas as pd

# Programm-Bibliothek zur schnellen numerischen 
# Bearbeitung von Tabellendaten und Matrizen.
import numpy as np

# Programm-Bibliothek zu grafischen und interaktiven 
# Darstellung von Daten
import altair as alt
alt.renderers.enable('notebook')

# Programm-Bibliothek u.a. Lineare Regression
from sklearn.linear_model import LinearRegression

## Laden und erstes Explorieren der Daten *(Gathering, First Exploration)*

### Laden und erstes Explorieren der Verkehrsdaten

In [11]:
# Laden der 'Verkehrsdaten' 
# aus Datei'Fremont_Bridge_Hourly_Bicycle_Counts_by_Month_October_2012_to_present.csv' 
# im Ordner 'Daten'
Verkehrsdaten = pd.read_csv(
    Pfad_zu_Daten + 'Fremont_Bridge_Hourly_Bicycle_Counts_by_Month_October_2012_to_present.csv')

In [12]:
# Lesen der obersten Reihe der 'Verkehrsdaten'
Verkehrsdaten.head()

,Date,Fremont Bridge East Sidewalk,Fremont Bridge West Sidewalk
0,05/31/2019 11:00:00 PM,22.0,35.0
1,05/31/2019 10:00:00 PM,29.0,52.0
2,05/31/2019 09:00:00 PM,46.0,57.0
3,05/31/2019 08:00:00 PM,48.0,92.0
4,05/31/2019 07:00:00 PM,97.0,151.0


In [ ]:
# Lesen der untersten Reihen der 'Verkehrsdaten'
Verkehrsdaten.tail()

In [ ]:
# Abstrakte Information über die 'Verkehrsdaten'
Verkehrsdaten.info()

In [ ]:
# Grundlegende Statistik der 'Verkehrsdaten'
Verkehrsdaten.describe(include='all').T

### *Do-It-Yourself: Laden und erstes Explorieren der Wetterdaten*

In [ ]:
# Lesen der 'Wetterdaten' 
# aus Datei '178440.csv' 
# im Ordner 'Daten' 
Wetterdaten = pd.read_csv('Daten/1786440.csv')

In [ ]:
# Lesen der obersten Reihe der 'Wetterdaten'
Wetterdaten.head()

In [ ]:
# Lesen der untersten Reihe der 'Wetterdaten'
Wetterdaten.tail()

In [ ]:
# Abstrakte Information über die 'Wetterdaten'
Wetterdaten.info()

In [ ]:
# Grundlegende Statistik der 'Wetterdaten'
Wetterdaten.describe(include='all').T

## Strukturieren der Daten *(Preprocessing)*

In [ ]:
# Anlegen einer neuen Datenstruktur 'Daten'
Daten = pd.DataFrame()

### Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten

Zunächst formatieren wir die Spalte 'Date' von 'verkehrsdaten' in ein Format um, dass von 'pandas' als Zeiten interpretiert werden kann.

In [ ]:
# Konvertieren der Spalte 'Date' in 'Verkehrsdaten'
# in ein "echtes" Datumsformat
Verkehrsdaten['Date'] = pd.to_datetime(Verkehrsdaten['Date'])

Dann setzen wir den Index der Tabelle 'verkehrsdaten' auf 'date'

In [ ]:
# Ersetzen der Index-Spalte mit der Spalte 'Date' aus 'Verkehrsdaten'
Verkehrsdaten = Verkehrsdaten.set_index('Date')

Um Vergleichbarkeit zu den Wetterdaten zu erreichen, aggregieren wir 

In [ ]:
# Aggregieren und Ersetzen der stündlichen 'Verkehrsdaten' 
# durch tägliche 'Verkehrsdaten'
Verkehrsdaten = Verkehrsdaten.resample('d').sum()

In [ ]:
# Lesen von Beispielreihen der 'Verkehrsdaten'
Verkehrsdaten.sample(5)

In [ ]:
# Abstrakte Information über 'Verkehrsdaten'
Verkehrsdaten.info()

In [ ]:
# Grundlegende Statistik der 'Verkehrsdaten'
Verkehrsdaten.describe(include='all').T

In [ ]:
# Übertragen der Summe der Spalten aus den 'Verkehrsdaten'
# in eine (neue) Spalte 'Fahrten' der 'Daten'
Daten['Fahrten'] = Verkehrsdaten['Fremont Bridge East Sidewalk'] + Verkehrsdaten['Fremont Bridge West Sidewalk']

In [ ]:
# Benennung des Index der 'Daten' mit 'Datum'
Daten.index = pd.DatetimeIndex(data=Verkehrsdaten.index, name='Datum')

In [ ]:
# Lesen von Beispielreihen aus den 'Daten'
Daten.sample(5)

### Ermitteln und Hinzufügen berechneter Daten (Tageslänge und Wochentag)

In [ ]:
# Anlegen einer neuen Datenstruktur 'Berechnete_Daten'
# mit Übernahme der Index-Struktur von 'Daten'
Berechnete_Daten = pd.DataFrame(index=Daten.index)

In [ ]:
# Neue Spalte 'Tag' in 'Berechnete_Daten'
# aus den Zahlenwerten des 'index' (der das Datum enthalt)
Berechnete_Daten['Tag'] = pd.to_numeric(Berechnete_Daten.index)

In [ ]:
# Lesen von Beispielsreihen aus 'Berechnete Daten'
Berechnete_Daten['Tag'].sample(5)

In [ ]:
# Umwandeln der Spalte 'Tag' in ganzzahlige Werte
# die einzelnen Tagen entsprechen
Tageslaenge_in_Nanosekunden = 24*60*60*1_000_000_000
Berechnete_Daten['Tag'] = (Berechnete_Daten['Tag'] - Berechnete_Daten['Tag'].min()) \
                        / Tageslaenge_in_Nanosekunden

In [ ]:
# Lesen der ersten Reihen von 'Berechnete Daten'
Berechnete_Daten.head(5)

In [ ]:
# Formel zur Berechnung der Tageslänge in Abhängigkeit des
# Datums und des Laengengrades 
# (voreingestellt ist der Längengrad von Seattle)

def Berechne_Tageslaenge(Datum, Laengengrad=47.61):
    
    # Neigung der Erdachse in Grad
    Neigung_erdachse = 23.44
    
    # Wintersonnenwende
    Wintersonnenwende = pd.datetime(2000, 12, 21)
    
    # Tage seit Wintersonnenwende (WSW)
    Zeit_seit_WSW = Datum - Wintersonnenwende
    Tage_seit_WSW = Zeit_seit_WSW.total_seconds() / (24. * 60. * 60.)
    Tage_seit_WSW = Tage_seit_WSW % 365.25
    
    m = 1. - \
        np.tan(np.radians(Laengengrad)) * \
        np.tan(
            np.radians(Neigung_erdachse) * 
            np.cos(Tage_seit_WSW * np.pi / 182.625) 
    )
    
    m = max(0, min(m, 2))
    
    Tageslaenge = 24. * np.degrees(np.arccos(1 - m)) / 180.
    
    return Tageslaenge

In [ ]:
# Aufbau einer Liste von Tageslaengen und 
# Übernahme in eine neue Spalte 'Tageslaenge'
# in 'Berechnete_Daten'
Tageslaengen = list(map(Berechne_Tageslaenge, Berechnete_Daten.index))
Berechnete_Daten['Tageslaenge'] = Tageslaengen

In [ ]:
# Lesen von Beispielreihen aus 'Berechnete Daten'
Berechnete_Daten.sample(5)

In [ ]:
# Berechnung, ob ein Tag ein Werktag 
# ist (1) oder nicht (0) und
# Übernahme in eine neue Spalte 'ist_Werktag' von 'Berechnete_Daten'
Berechnete_Daten['ist_Werktag'] = 1 - Berechnete_Daten.index.dayofweek // 5

In [ ]:
# Lesen von Beispielreihen aus 'Berechnete Daten'
Berechnete_Daten.sample(5)

In [ ]:
# Übernahme von 'Tageslaenge' und 'ist_Werktag'
# aus 'Berechnete_Daten' in die 'Daten'
Daten[['Tag', 'Tageslaenge', 'ist_Werktag']] = Berechnete_Daten[['Tag', 'Tageslaenge', 'ist_Werktag']]

In [ ]:
# Lesen von Beispielreihen aus 'Daten'
Daten.sample(5)

### *Do-It-Yourself: Formatieren und Hinzufügen ausgewählter Wetterdaten*

In [ ]:
# Konvertieren der Spalte 'DATE' in 'Wetterdaten'
# in ein "echtes" Datumsformat
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Wetterdaten['DATE'] = pd.to_datetime(Wetterdaten['DATE'])

In [ ]:
# Ersetzen der Index-Spalte mit der Spalte 'DATE' aus 'Wetterdaten'
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Wetterdaten = Wetterdaten.set_index('DATE')

In [ ]:
# Abstrakte Information über 'Wetterdaten'
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Wetterdaten.info()

In [ ]:
# Grundlegende Statistik der 'Wetterdaten'
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Wetterdaten.describe(include='all').T

In [ ]:
# Übernahme der Spalten 'TAVG' (average temperature = durchschnittliche Temperatur)
# und 'PRCP' (precipitation = Niederschlag) in 
# neue Spalten von 'Daten' 'Temperatur' und 'Niederschlag'
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Daten['Temperatur'] = Wetterdaten['TAVG']
Daten['Niederschlag'] = Wetterdaten['PRCP']

In [ ]:
# Lesen von Beispielreihen aus 'Daten'
# (vgl. 'Formattieren und Aggregieren und Hinzufügen der Verkehrsdaten')
Daten.sample(5)

## Bereinigung und weitere Exploration der Daten *(Cleansing, Exploration)*

In [ ]:
# Kopieren von 'Daten' in eine
# neue Datentabelle 'DatenQuelle'
# um die Daten vor weiterer Veränderung zu schützen
DatenQuelle = Daten.copy()
DatenQuelle.sample()

In [ ]:
# Neue Spalte 'Datum' aus dem 
# Index von DatenQuelle (und
# Erzeugen eines neuen, generischen Indexes)
DatenQuelle.reset_index(drop=False, inplace=True)

In [ ]:
# Lesen von Beispielreihen von 'DatenQuelle'
DatenQuelle.sample(5)

In [ ]:
# Reorganisation der Spaltenreihenfolge
DatenQuelle = DatenQuelle[['Datum', 'Tag', 'Fahrten', 'Tageslaenge', 'ist_Werktag', 'Temperatur', 'Niederschlag']]

In [ ]:
# Grundlegende Statistik von 'DatenQuelle'
DatenQuelle.describe(include='all').T

In [ ]:
# Löschen aller Reihen die 
# keine Werte in einer oder mehreren Spalten enthalten
DatenQuelle.dropna(inplace=True)

In [ ]:
# Grundlegende Statistik von 'DatenQuelle'
DatenQuelle.describe(include='all').T

In [ ]:
# Funktion zur Erzeugung
# einer Grafik für eine Variable
# im zeitlichen Verlauf
def Grafik_Verlauf(Abhaengige_Variable='Fahrten', 
                   DatenQuelle=DatenQuelle, 
                   # Variablen für die optionale 
                   # weitere Variation der Grafiken
                   # (wird später benutzt)
                   Glaettungsfenster=None, 
                   Auswahlfilter=None):
    
    if Glaettungsfenster:
        
        Grafik_Verlauf = alt.Chart(
            data=DatenQuelle,
            width=600, height=300
        ).mark_line(
            color='orange', size=2
        ).transform_window(
            Gelaettete_Variable='mean('+ Abhaengige_Variable + ')',
            frame=[0, Glaettungsfenster]
        ).encode(
            x=alt.X('Datum:T',
                    axis=alt.Axis(title=None)),
            y=alt.Y('Gelaettete_Variable:Q',
                    axis=alt.Axis(title=None)
                   )
        ).properties(
            title='Zeitlicher Verlauf von ' + Abhaengige_Variable + \
            ' (inkl. ' + str(Glaettungsfenster) + '-Tage-Glaettung)'
        )
        
    else:
        
        Grafik_Verlauf = alt.Chart(
            data=DatenQuelle,
            width=600, height=300
        ).mark_point(
            size=2
        ).encode(
            x=alt.X('Datum',
                    axis=alt.Axis(title=None)),
            y=alt.Y(Abhaengige_Variable,
                    axis=alt.Axis(title=None)),
            tooltip=list(DatenQuelle.columns)
        ).properties(
            title='Zeitlicher Verlauf von ' + Abhaengige_Variable
        )
        
        if Auswahlfilter:
            
            Grafik_Verlauf = Grafik_Verlauf.encode(
                color=alt.condition(Auswahlfilter,
                                    alt.ColorValue('#1f77b4'), 
                                    alt.ColorValue('lightgrey'))
            ).add_selection(Auswahlfilter)
            
        Trendvariable = 'Trend_' + Abhaengige_Variable + '_iAv_Tag'
        
        if Trendvariable in DatenQuelle.columns:
            
            Grafik_Trend = alt.Chart(
                data=DatenQuelle
            ).mark_line(
                color='red', size=2
            ).encode(
                x=alt.X('Datum'),
                y=alt.X(Trendvariable)
            )
            
            Grafik_Verlauf = Grafik_Verlauf + Grafik_Trend
        
    return Grafik_Verlauf

In [ ]:
Grafik_Verlauf()

In [ ]:
Grafik_Verlauf(Glaettungsfenster=7)

In [ ]:
Grafik_Verlauf(Glaettungsfenster=7) + Grafik_Verlauf()

In [ ]:
def Grafik_Streuung(Unabhaengige_Variable, 
                    Abhaengige_Variable='Fahrten',
                    DatenQuelle=DatenQuelle,
                    Auswahlfilter=None):
    
    Grafik_Streuung = alt.Chart(
        data=DatenQuelle,
        width=263, height=263
    ).mark_point(
        size=1
    ).encode(
        x=alt.X(Unabhaengige_Variable,
                scale=alt.Scale(zero=False)),
        y=alt.Y(Abhaengige_Variable,
                scale=alt.Scale(zero=False)),
        tooltip=list(DatenQuelle.columns)
    )
    
    if Auswahlfilter:
        
        Grafik_Streuung = Grafik_Streuung.encode(
            color=alt.condition(Auswahlfilter,
                                alt.ColorValue('#1f77b4'), alt.ColorValue('lightgrey'))
        ).add_selection(Auswahlfilter)
        
    Trendvariable = 'Trend_' + Abhaengige_Variable + '_iAv_' + Unabhaengige_Variable
    
    if Trendvariable in DatenQuelle.columns:
        
        Grafik_Trend = alt.Chart(
            data=DatenQuelle
        ).mark_line(
            color='red', size=2
        ).encode(
            x=alt.X(Unabhaengige_Variable),
            y=alt.Y(Trendvariable,
                    axis=alt.Axis(title=Abhaengige_Variable + ' (mit Trendlinie)'))
        )
        
        Grafik_Streuung = Grafik_Streuung + Grafik_Trend
    
    return Grafik_Streuung

In [ ]:
Grafik_Streuung('Tageslaenge')

In [ ]:
def Grafik_Uebersicht(Abhaengige_Variable='Fahrten',
                      DatenQuelle=DatenQuelle):
    
    Auswahlfilter = alt.selection(type='interval', resolve='global')
    
    Oben = Grafik_Verlauf(Abhaengige_Variable=Abhaengige_Variable, 
                          DatenQuelle=DatenQuelle, 
                          Glaettungsfenster=7) \
         + Grafik_Verlauf(Abhaengige_Variable=Abhaengige_Variable,
                          DatenQuelle=DatenQuelle,
                          Auswahlfilter=Auswahlfilter)
    
    Links = Grafik_Streuung('Tageslaenge',
                            Abhaengige_Variable=Abhaengige_Variable, 
                            DatenQuelle=DatenQuelle,                            
                            Auswahlfilter=Auswahlfilter) \
          & Grafik_Streuung('ist_Werktag',
                            Abhaengige_Variable=Abhaengige_Variable, 
                            DatenQuelle=DatenQuelle,
                            Auswahlfilter=Auswahlfilter)
    
    Rechts = Grafik_Streuung('Temperatur',
                            Abhaengige_Variable=Abhaengige_Variable, 
                            DatenQuelle=DatenQuelle,
                            Auswahlfilter=Auswahlfilter) & \
             Grafik_Streuung('Niederschlag',
                            Abhaengige_Variable=Abhaengige_Variable, 
                            DatenQuelle=DatenQuelle,
                            Auswahlfilter=Auswahlfilter)
    
    Grafik_Uebersicht = Oben & (Links | Rechts)
    
    return Grafik_Uebersicht

In [ ]:
Grafik_Uebersicht()    

## Modellierung der Daten *(Modelling)*

### Lineare Regression - eine kurze Einführung

In [ ]:

<a href="https://www.youtube.com/watch?v=nk2CQITm_eo" target="_blank"><img src="http://img.youtube.com/vi/nk2CQITm_eo/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="600" height="400" border="0" /></a>

In [ ]:
def LR_Modell_und_Daten(Liste_Unabhaengige_Regressionsvariablen,
                        Abhaengige_Regressionsvariable='Fahrten',
                        DatenQuelle=DatenQuelle):
    
    Daten_LR_Modell = DatenQuelle.copy()
    
    Daten_Unabhaengige_Regressionsvariablen = Daten_LR_Modell[Liste_Unabhaengige_Regressionsvariablen]
    Daten_Abhaengige_Regressionsvariable = Daten_LR_Modell[Abhaengige_Regressionsvariable]
    
    LR_Modell = LinearRegression(
    ).fit(Daten_Unabhaengige_Regressionsvariablen, 
          Daten_Abhaengige_Regressionsvariable)
    
    Daten_LR_Modell['Trend'] = LR_Modell.predict(Daten_Unabhaengige_Regressionsvariablen)
    
    Daten_LR_Modell[Abhaengige_Regressionsvariable+'_trendbereinigt']\
    = Daten_LR_Modell[Abhaengige_Regressionsvariable] \
    - Daten_LR_Modell['Trend'] \
    + Daten_LR_Modell['Trend'].mean()
    
    for Unabhaengige_Ceteris_Paribus_Regressionsvariable \
    in Liste_Unabhaengige_Regressionsvariablen:
        
        Ceteris_Paribus_Regressionsdatensatz = pd.DataFrame(index=Daten_LR_Modell.index)
        
        for Unabhaengige_Regressionsvariable in Liste_Unabhaengige_Regressionsvariablen:
            
            if Unabhaengige_Regressionsvariable == Unabhaengige_Ceteris_Paribus_Regressionsvariable:
                
                Ceteris_Paribus_Regressionsdatensatz[Unabhaengige_Ceteris_Paribus_Regressionsvariable] \
                = DatenQuelle[Unabhaengige_Ceteris_Paribus_Regressionsvariable]
                
            else:
                
                Ceteris_Paribus_Regressionsdatensatz[Unabhaengige_Regressionsvariable] \
                = DatenQuelle[Unabhaengige_Regressionsvariable].mean()
                
        Daten_LR_Modell['Trend_' + Abhaengige_Regressionsvariable \
                        + '_iAv_'+ Unabhaengige_Ceteris_Paribus_Regressionsvariable] \
        = LR_Modell.predict(Ceteris_Paribus_Regressionsdatensatz)
        
        Daten_LR_Modell['Trend_' + Abhaengige_Regressionsvariable + '_trendbereinigt'\
                        + '_iAv_'+ Unabhaengige_Ceteris_Paribus_Regressionsvariable] \
        = Daten_LR_Modell['Trend_' + Abhaengige_Regressionsvariable \
                        + '_iAv_'+ Unabhaengige_Ceteris_Paribus_Regressionsvariable].mean()
        
    
    return LR_Modell, Daten_LR_Modell

### Ein einfaches lineares Modell - Abhängigkeit der Fahrten von der Nummer des Tages

In [ ]:
Liste_Unabhaengige_Regressionsvariablen = [
    'Tag',
]

LR_Modell, Daten_LR_Modell = LR_Modell_und_Daten(
    Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen,
    Abhaengige_Regressionsvariable='Fahrten',
    DatenQuelle=DatenQuelle)

In [ ]:
Daten_LR_Modell.sample(5)

In [ ]:
Grafik_Uebersicht(Abhaengige_Variable='Fahrten', 
                  DatenQuelle=Daten_LR_Modell)

In [ ]:
Grafik_Uebersicht(Abhaengige_Variable='Fahrten_trendbereinigt', 
                  DatenQuelle=Daten_LR_Modell)

In [ ]:
def Statistik(Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen,
              Abhaengige_Regressionsvariable='Fahrten', 
              Daten_LR_Modell=Daten_LR_Modell, 
              LR_Modell=LR_Modell):
    
    Steigung = LR_Modell.coef_
    
    y = Daten_LR_Modell[Abhaengige_Regressionsvariable]
    y_trend = Daten_LR_Modell['Trend']
    X = Daten_LR_Modell[Liste_Unabhaengige_Regressionsvariablen]
    
    var_y = np.sum((y - y_trend) ** 2) / len(y)
    X2 = np.hstack([X, np.ones((X.shape[0], 1))])
    C = var_y * np.linalg.inv(np.dot(X2.T, X2))
    var = C.diagonal()
    
    Fehler = np.sqrt(var[:])
    
    for Zaehler, Unabhaengige_Regressionsvariable \
    in enumerate(Liste_Unabhaengige_Regressionsvariablen):
        
        print('{0:.2f} +/- {1:.2f}'.format(Steigung[Zaehler], Fehler[Zaehler]), 
              Abhaengige_Regressionsvariable, 
              'je Einheit', 
              Unabhaengige_Regressionsvariable)
              
    return None

In [ ]:
Statistik(Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen, 
          Abhaengige_Regressionsvariable='Fahrten', 
          Daten_LR_Modell=Daten_LR_Modell, 
          LR_Modell=LR_Modell)

In [ ]:
def Lineare_Regressionsanalyse(
    Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen,
    Abhaengige_Regressionsvariable='Fahrten',
    DatenQuelle=DatenQuelle):
    
    LR_Modell, Daten_LR_Modell = LR_Modell_und_Daten(
        Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen,
        Abhaengige_Regressionsvariable=Abhaengige_Regressionsvariable,
        DatenQuelle=DatenQuelle)
    
    Grafik_Uebersicht_normal = \
    Grafik_Uebersicht(Abhaengige_Variable=Abhaengige_Regressionsvariable, 
                      DatenQuelle=Daten_LR_Modell)
    
    Grafik_Uebersicht_trendbereinigt = \
    Grafik_Uebersicht(Abhaengige_Variable=Abhaengige_Regressionsvariable + '_trendbereinigt', 
                      DatenQuelle=Daten_LR_Modell)
    
    Statistik(Liste_Unabhaengige_Regressionsvariablen=Liste_Unabhaengige_Regressionsvariablen,
              Abhaengige_Regressionsvariable=Abhaengige_Regressionsvariable, 
              Daten_LR_Modell=Daten_LR_Modell, LR_Modell=LR_Modell)
    
    return Grafik_Uebersicht_normal & Grafik_Uebersicht_trendbereinigt

In [ ]:
Lineare_Regressionsanalyse(['Tag'])

### *Do-It-Yourself: Abhängigkeit der Fahrten von der Temperatur*

In [ ]:
Lineare_Regressionsanalyse(['Temperatur'])

### Ein multi-lineares Modell - Abhängigkeit der Fahrten von Tageslänge und ist_Werktag

In [ ]:
Lineare_Regressionsanalyse(['Tageslaenge', 'ist_Werktag'])

### *Do-It-Yourself: Abhängigkeit der Fahrten von Temperatur und Niederschlag*

### Das vollständige multi-lineare Modell

In [ ]:
Lineare_Regressionsanalyse(['Tageslaenge', 'ist_Werktag', 'Temperatur', 'Niederschlag'])

In [ ]:
Lineare_Regressionsanalyse(['Tageslaenge', 'ist_Werktag', 'Temperatur', 'Niederschlag', 'Tag'])

## Ergebnisse